In [2]:
!pip install -qU transformers
#! pip install accelerate

In [3]:
from torch import cuda
import transformers

In [4]:
cuda.is_available()

False

In [5]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "drive/MyDrive/project_cs685/dataset/test_dataset.csv"

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
generate_text = transformers.pipeline(
    model='NousResearch/Llama-2-7b-chat-hf',
    low_cpu_mem_usage=True,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    #device=1,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [9]:
%time
res = generate_text("Explain to me the difference between nuclear fission and fusion.")

print(res[0]["generated_text"])

CPU times: user 4 µs, sys: 7 µs, total: 11 µs
Wall time: 12.9 µs


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Explain to me the difference between nuclear fission and fusion.
Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This typically occurs when an atom is bombarded with a high-energy particle, such as a neutron. The most commonly used fuel for nuclear fission is uranium-235 (U-235), which has a relatively simple structure consisting of 92 protons and 146 neutrons. When U-235 is hit by a neutron, it undergoes a chain reaction, causing the nucleus to split into two smaller nuclei, releasing a large amount of energy in the form of heat and radiation.
Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but unlike fission, it does not produce any long-


In [4]:
import pandas as pd

#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
# The instruction dataset to use
dataset_name = "drive/MyDrive/project_cs685/dataset/test_dataset.csv"
test_dataset_df = pd.read_csv(dataset_name)


In [3]:
instruction_text = "Refer to the given context to respond the question with either 'yes' or 'no'."

#"If you're unsure of the answer, simply state 'na' without guessing."
#"Use the following context to answer the question. The answer will be 'yes' or 'no'. If you don't know the answer, just say 'na', don't try to make up an answer."


def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{instruction_text}"
    input_context = f"{INPUT_KEY}\n{sample['background']} {sample['given_info']} {sample['question']}" #if sample["input"] else None
    response = f"{RESPONSE_KEY}\n" # "{sample['answer']}"
    #end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response] if part] # response, end

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)
    formatted_prompt = f"<s>[INST] {formatted_prompt} [/INST]"

    # Store the formatted prompt template in a new key "text"
    #sample["text"] = formatted_prompt

    return formatted_prompt


In [6]:
test_dataset_df['formatted_prompt'] = test_dataset_df.apply(create_prompt_formats, axis=1)

In [7]:
test_dataset_df.head()

,question_id,desc_id,given_info,question,answer,meta,reasoning,background,formatted_prompt
0,3888,nonsense13-frontdoor-marginal-modelNone-spec5-q1,The overall probability of fritz is 10%. For t...,Is zibbo less likely than not zibbo overall?,yes,"{'story_id': 'nonsense13', 'graph_id': 'frontd...",{'step0': 'Let V1 = glopp; X = fritz; V3 = flu...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...
1,8401,nonsense17-frontdoor-nie-modelNone-spec8-q0,"For those who are not bleen, the probability o...",Does bleen positively affect zibby through flarn?,no,"{'story_id': 'nonsense17', 'graph_id': 'frontd...",{'step0': 'Let V1 = quop; X = bleen; V3 = flar...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...
2,8132,nonsense17-confounding-ate-modelNone-spec7-q1,"For those who are not flarn and are not bleen,...",Will bleen decrease the chance of zibby?,no,"{'story_id': 'nonsense17', 'graph_id': 'confou...",{'step0': 'Let V1 = flarn; X = bleen; Y = zibb...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...
3,4701,nonsense14-confounding-ate-modelNone-spec0-q0,For those who are not flurnt and are not plizz...,Will plizz increase the chance of brifft?,no,"{'story_id': 'nonsense14', 'graph_id': 'confou...",{'step0': 'Let V1 = flurnt; X = plizz; Y = bri...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...
4,7168,nonsense16-mediation-ate-modelNone-spec9-q1,"For those who are not gruntz, the probability ...",Will gruntz decrease the chance of flurrk?,no,"{'story_id': 'nonsense16', 'graph_id': 'mediat...",{'step0': 'Let X = gruntz; V2 = zowpxo; Y = fl...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...


In [13]:
! pip install datasets

In [14]:
from datasets import Dataset

In [15]:
# Load dataset
dataset = Dataset.from_pandas(test_dataset_df[['question_id', 'desc_id', 'given_info', 'question', 'answer',\
                                               'background', 'formatted_prompt']], split="test")

Map:   0%|          | 0/2464 [00:00<?, ? examples/s]

In [16]:
head_dataset = dataset.select(range(5))

In [17]:
def generate_response(prompt):
  res = generate_text(prompt)
  return res[0]["generated_text"]

In [ ]:
head_dataset = head_dataset.map(lambda x: {"llama2_response": generate_response(x["text"])}, batched=True)

In [ ]:
for i in head_dataset.select(range(2)):
  print(i['llama2_ft_response'][0]['generated_text'])
  print("-----------------------")

In [1]:
! pip install together

In [ ]:
import os
from together import Together

client = Together(api_key="API KEY")

In [14]:
def generate_response(prompt):
  try:
    response = client.chat.completions.create(
      model="meta-llama/Llama-2-7b-chat-hf", #"meta-llama/Llama-3-8b-chat-hf",
      messages=[{"role": "user", "content": f"{prompt}"}],)
    return response.choices[0].message.content
  except:
    return None

In [11]:
test_dataset_df["llama2_response"] = test_dataset_df.formatted_prompt.apply(generate_response)

<ipython-input-11-b0ffd20da5b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_df1["llama2_response"] = test_dataset_df1.formatted_prompt.apply(generate_response)


In [12]:
test_dataset_df.head()

,question_id,desc_id,given_info,question,answer,meta,reasoning,background,formatted_prompt,llama2_response
0,3888,nonsense13-frontdoor-marginal-modelNone-spec5-q1,The overall probability of fritz is 10%. For t...,Is zibbo less likely than not zibbo overall?,yes,"{'story_id': 'nonsense13', 'graph_id': 'frontd...",{'step0': 'Let V1 = glopp; X = fritz; V3 = flu...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,Yes.
1,8401,nonsense17-frontdoor-nie-modelNone-spec8-q0,"For those who are not bleen, the probability o...",Does bleen positively affect zibby through flarn?,no,"{'story_id': 'nonsense17', 'graph_id': 'frontd...",{'step0': 'Let V1 = quop; X = bleen; V3 = flar...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,Yes.
2,8132,nonsense17-confounding-ate-modelNone-spec7-q1,"For those who are not flarn and are not bleen,...",Will bleen decrease the chance of zibby?,no,"{'story_id': 'nonsense17', 'graph_id': 'confou...",{'step0': 'Let V1 = flarn; X = bleen; Y = zibb...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,Yes.
3,4701,nonsense14-confounding-ate-modelNone-spec0-q0,For those who are not flurnt and are not plizz...,Will plizz increase the chance of brifft?,no,"{'story_id': 'nonsense14', 'graph_id': 'confou...",{'step0': 'Let V1 = flurnt; X = plizz; Y = bri...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,Yes.
4,7168,nonsense16-mediation-ate-modelNone-spec9-q1,"For those who are not gruntz, the probability ...",Will gruntz decrease the chance of flurrk?,no,"{'story_id': 'nonsense16', 'graph_id': 'mediat...",{'step0': 'Let X = gruntz; V2 = zowpxo; Y = fl...,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,Yes.


In [15]:
test_dataset_df.shape

(2464, 9)

In [29]:
def process_llama2_response(response_text):
  response_text_lst = response_text.split(' ')
  if len(response_text_lst) > 0:
    for i in response_text_lst:
      if 'yes' in i.lower():
        return 'yes'
      elif 'no' in i.lower():
        return 'no'
  else:
    return 'na'

In [30]:
test_dataset_df["llama2_response_processed"] = test_dataset_df.llama2_response.apply(process_llama2_response)
test_dataset_df.to_csv('drive/MyDrive/project_cs685/model/llama-2/results/output_data.csv', index=False, header = True)

In [31]:
test_dataset_df.groupby(["answer","llama2_response_processed"])["question_id"].count()

answer  llama2_response_processed
no      no                             17
        yes                          1288
yes     no                              7
        yes                          1152
Name: question_id, dtype: int64